In [1]:
import os
import random
from functools import partial
from typing import List

import torch
import torch.nn as nn

import numpy as np
import albumentations as A

In [2]:
from torch.utils.data import DataLoader
from resized_dataset import make_dataset
from setseed import set_seed

BATCH_SIZE = 8
resize = 512
RANDOM_SEED = 21

set_seed(RANDOM_SEED)

train_dataset, valid_dataset = make_dataset(RANDOM_SEED = RANDOM_SEED)

train_loader = DataLoader(
    dataset=train_dataset, 
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=6,
    drop_last=True,
)

valid_loader = DataLoader(
    dataset=valid_dataset, 
    batch_size=2,
    shuffle=False,
    num_workers=2,
    drop_last=False
)

In [3]:
import timm
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.encoders._base import EncoderMixin

class hrnet_encoder(nn.Module, EncoderMixin):
    def __init__(self, **kwargs):
        super().__init__()
        self.net = timm.create_model('hrnet_w18', pretrained=False, num_classes=29)

    def forward(self, x):
        out = self.net(x)

        return out

smp.encoders.encoders["hrnet_w18"] = {
    "encoder": hrnet_encoder, # encoder class here
    "pretrained_settings": {
        "imagenet": {
            "mean": [0.485, 0.456, 0.406],
            "std": [0.229, 0.224, 0.225],
            "url": 'https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-hrnet/hrnetv2_w18-8cb57bb9.pth',
            "input_space": "RGB",
            "input_range": [0, 1],
        },
    },
    "params": {
        "pretrain_img_size": 224,
        "crop_pct": 0.95
    }
}

In [4]:
set_seed(RANDOM_SEED)

model = smp.DeepLabV3Plus(
    encoder_name="resnet50",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=29,                      # model output channels (number of classes in your dataset)
)
model.name = 'DeepLabV3Plus_resnet50'

In [5]:
LR = 1e-4
epoch = 50
VAL_EVERY = 1 # validation 주기

In [6]:
import torch.optim as optim

set_seed(RANDOM_SEED)

optimizer = optim.Adam(params=model.parameters(), lr=LR, weight_decay=1e-6)

In [7]:
from loss import FocalLoss, DiceLoss

set_seed(RANDOM_SEED)

#criterion = nn.BCEWithLogitsLoss()
#criterion = nn.SmoothL1Loss()
criterion = [(nn.BCEWithLogitsLoss(), 0.75), (DiceLoss(), 0.25)]
#criterion = [(nn.BCEWithLogitsLoss(), 0.5), (nn.SmoothL1Loss(), 0.5)]

In [8]:
#모델 저장 경로 설정
try:
    folder_name = "[{}]_[size:{}]_[loss:{}]_[LR:{}]_[seed:{}]_[epoch:{}]".format(model.name, (resize, resize), criterion, LR, RANDOM_SEED, epoch)
except:
    folder_name = "[{}]_[size:{}]_[loss:{}]_[LR:{}]_[seed:{}]_[epoch:{}]".format("model", (resize, resize), criterion, LR, RANDOM_SEED, epoch)

In [9]:
from train import train

set_seed(RANDOM_SEED)

folder_name = train(model, train_loader, valid_loader, criterion, optimizer, epoch, VAL_EVERY, folder_name)

Start training..
result model will be saved in /opt/ml/input/code/trained_model/[DeepLabV3Plus_efficientnet-b4]_[size:(512, 512)]_[loss:[(BCEWithLogitsLoss(), 0.75), (DiceLoss(), 0.25)]]_[LR:0.0001]_[seed:21]_[epoch:50]


wandb: Currently logged in as: tjwlgns02 (sixseg). Use `wandb login --relogin` to force relogin


2023-06-09 17:33:02 | Epoch [1/50], Step [0/80]
2023-06-09 17:34:22 | Epoch [1/50], Step [25/80], Loss: 0.675
2023-06-09 17:35:41 | Epoch [1/50], Step [50/80], Loss: 0.5911
2023-06-09 17:36:58 | Epoch [1/50], Step [75/80], Loss: 0.5213
Start validation # 1



finger-1    : 0.0000
finger-2    : 0.0000
finger-3    : 0.0156
finger-4    : 0.0000
finger-5    : 0.0000
finger-6    : 0.0010
finger-7    : 0.1648
finger-8    : 0.0000
finger-9    : 0.0000
finger-10   : 0.0013
finger-11   : 0.2051
finger-12   : 0.0008
finger-13   : 0.0002
finger-14   : 0.0000
finger-15   : 0.0033
finger-16   : 0.0000
finger-17   : 0.0000
finger-18   : 0.0000
finger-19   : 0.0000
Trapezium   : 0.0036
Trapezoid   : 0.0000
Capitate    : 0.0696
Hamate      : 0.0356
Scaphoid    : 0.0052
Lunate      : 0.0897
Triquetrum  : 0.0000
Pisiform    : 0.0000
Radius      : 0.6005
Ulna        : 0.0707
Best performance at epoch: 1, 0.0000 -> 0.0437
Save model in /opt/ml/input/code/trained_model/[DeepLabV3Plus_efficientnet-b4]_[size:(512, 512)]_[loss:[(BCEWithLogitsLoss(), 0.75), (DiceLoss(), 0.25)]]_[LR:0.0001]_[seed:21]_[epoch:50]
2023-06-09 17:38:19 | Epoch [2/50], Step [0/80]
2023-06-09 17:39:35 | Epoch [2/50], Step [25/80], Loss: 0.4578
2023-06-09 17:40:51 | Epoch [2/50], Step [50/


finger-1    : 0.0000
finger-2    : 0.0000
finger-3    : 0.0000
finger-4    : 0.0000
finger-5    : 0.0000
finger-6    : 0.0000
finger-7    : 0.1005
finger-8    : 0.0000
finger-9    : 0.0000
finger-10   : 0.0000
finger-11   : 0.1016
finger-12   : 0.0000
finger-13   : 0.0000
finger-14   : 0.0000
finger-15   : 0.0000
finger-16   : 0.0000
finger-17   : 0.0000
finger-18   : 0.0000
finger-19   : 0.0000
Trapezium   : 0.0000
Trapezoid   : 0.0000
Capitate    : 0.0000
Hamate      : 0.0000
Scaphoid    : 0.0000
Lunate      : 0.0000
Triquetrum  : 0.0000
Pisiform    : 0.0000
Radius      : 0.6514
Ulna        : 0.0001
2023-06-09 17:43:29 | Epoch [3/50], Step [0/80]
2023-06-09 17:44:46 | Epoch [3/50], Step [25/80], Loss: 0.3602
2023-06-09 17:46:03 | Epoch [3/50], Step [50/80], Loss: 0.342
2023-06-09 17:47:18 | Epoch [3/50], Step [75/80], Loss: 0.327
Start validation # 3


...


finger-1    : 0.9403
finger-2    : 0.9629
finger-3    : 0.9586
finger-4    : 0.9307
finger-5    : 0.9455
finger-6    : 0.9698
finger-7    : 0.9673
finger-8    : 0.9366
finger-9    : 0.9583
finger-10   : 0.9741
finger-11   : 0.9621
finger-12   : 0.9265
finger-13   : 0.9468
finger-14   : 0.9653
finger-15   : 0.9574
finger-16   : 0.9075
finger-17   : 0.9250
finger-18   : 0.9589
finger-19   : 0.9640
Trapezium   : 0.8950
Trapezoid   : 0.8533
Capitate    : 0.9272
Hamate      : 0.9151
Scaphoid    : 0.9094
Lunate      : 0.9065
Triquetrum  : 0.9070
Pisiform    : 0.8522
Radius      : 0.9820
Ulna        : 0.9774
2023-06-09 21:32:10 | Epoch [47/50], Step [0/80]
2023-06-09 21:33:27 | Epoch [47/50], Step [25/80], Loss: 0.0119
2023-06-09 21:34:44 | Epoch [47/50], Step [50/80], Loss: 0.012
2023-06-09 21:36:01 | Epoch [47/50], Step [75/80], Loss: 0.012
Start validation #47



finger-1    : 0.9344
finger-2    : 0.9603
finger-3    : 0.9595
finger-4    : 0.9270
finger-5    : 0.9513
finger-6    : 0.9718
finger-7    : 0.9644
finger-8    : 0.9434
finger-9    : 0.9614
finger-10   : 0.9727
finger-11   : 0.9585
finger-12   : 0.9313
finger-13   : 0.9569
finger-14   : 0.9656
finger-15   : 0.9577
finger-16   : 0.9113
finger-17   : 0.9247
finger-18   : 0.9530
finger-19   : 0.9632
Trapezium   : 0.8962
Trapezoid   : 0.8526
Capitate    : 0.9209
Hamate      : 0.9109
Scaphoid    : 0.9175
Lunate      : 0.9001
Triquetrum  : 0.9109
Pisiform    : 0.8513
Radius      : 0.9815
Ulna        : 0.9766
2023-06-09 21:37:20 | Epoch [48/50], Step [0/80]
2023-06-09 21:38:39 | Epoch [48/50], Step [25/80], Loss: 0.0115
2023-06-09 21:39:56 | Epoch [48/50], Step [50/80], Loss: 0.0119
2023-06-09 21:41:14 | Epoch [48/50], Step [75/80], Loss: 0.0121
Start validation #48



finger-1    : 0.9333
finger-2    : 0.9617
finger-3    : 0.9661
finger-4    : 0.9356
finger-5    : 0.9506
finger-6    : 0.9679
finger-7    : 0.9666
finger-8    : 0.9457
finger-9    : 0.9549
finger-10   : 0.9694
finger-11   : 0.9595
finger-12   : 0.9379
finger-13   : 0.9504
finger-14   : 0.9627
finger-15   : 0.9553
finger-16   : 0.9105
finger-17   : 0.9228
finger-18   : 0.9517
finger-19   : 0.9615
Trapezium   : 0.9110
Trapezoid   : 0.8618
Capitate    : 0.9204
Hamate      : 0.9161
Scaphoid    : 0.9264
Lunate      : 0.8946
Triquetrum  : 0.9142
Pisiform    : 0.8504
Radius      : 0.9821
Ulna        : 0.9777
2023-06-09 21:42:33 | Epoch [49/50], Step [0/80]
2023-06-09 21:43:50 | Epoch [49/50], Step [25/80], Loss: 0.0115
2023-06-09 21:45:06 | Epoch [49/50], Step [50/80], Loss: 0.0117
2023-06-09 21:46:21 | Epoch [49/50], Step [75/80], Loss: 0.0118
Start validation #49



finger-1    : 0.9383
finger-2    : 0.9593
finger-3    : 0.9685
finger-4    : 0.9401
finger-5    : 0.9431
finger-6    : 0.9636
finger-7    : 0.9666
finger-8    : 0.9435
finger-9    : 0.9547
finger-10   : 0.9678
finger-11   : 0.9606
finger-12   : 0.9370
finger-13   : 0.9497
finger-14   : 0.9628
finger-15   : 0.9535
finger-16   : 0.9094
finger-17   : 0.9286
finger-18   : 0.9557
finger-19   : 0.9557
Trapezium   : 0.9088
Trapezoid   : 0.8583
Capitate    : 0.9096
Hamate      : 0.8941
Scaphoid    : 0.9275
Lunate      : 0.9027
Triquetrum  : 0.9078
Pisiform    : 0.8471
Radius      : 0.9826
Ulna        : 0.9785
2023-06-09 21:47:41 | Epoch [50/50], Step [0/80]
2023-06-09 21:49:00 | Epoch [50/50], Step [25/80], Loss: 0.0114
2023-06-09 21:50:15 | Epoch [50/50], Step [50/80], Loss: 0.0113
2023-06-09 21:51:31 | Epoch [50/50], Step [75/80], Loss: 0.0118
Start validation #50



finger-1    : 0.9423
finger-2    : 0.9588
finger-3    : 0.9645
finger-4    : 0.9437
finger-5    : 0.9567
finger-6    : 0.9728
finger-7    : 0.9631
finger-8    : 0.9445
finger-9    : 0.9600
finger-10   : 0.9728
finger-11   : 0.9575
finger-12   : 0.9382
finger-13   : 0.9556
finger-14   : 0.9670
finger-15   : 0.9531
finger-16   : 0.9108
finger-17   : 0.9298
finger-18   : 0.9599
finger-19   : 0.9597
Trapezium   : 0.8998
Trapezoid   : 0.8497
Capitate    : 0.9070
Hamate      : 0.8910
Scaphoid    : 0.9194
Lunate      : 0.8978
Triquetrum  : 0.9068
Pisiform    : 0.8512
Radius      : 0.9802
Ulna        : 0.9784


In [10]:
from inference import inference

inference(folder_name, A.Resize(resize, resize))

In [11]:
set_seed(RANDOM_SEED)

model = smp.DeepLabV3Plus(
    encoder_name="timm-efficientnet-b4",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=29,                      # model output channels (number of classes in your dataset)
)
model.name = 'DeepLabV3Plus_efficientnet-b4'

optimizer = optim.Adam(params=model.parameters(), lr=LR, weight_decay=1e-6)

criterion = [(nn.BCEWithLogitsLoss(), 0.5), (nn.SmoothL1Loss(), 0.5)]

In [12]:
#모델 저장 경로 설정
try:
    folder_name = "[{}]_[size:{}]_[loss:{}]_[LR:{}]_[seed:{}]_[epoch:{}]".format(model.name, (resize, resize), criterion, LR, RANDOM_SEED, epoch)
except:
    folder_name = "[{}]_[size:{}]_[loss:{}]_[LR:{}]_[seed:{}]_[epoch:{}]".format("model", (resize, resize), criterion, LR, RANDOM_SEED, epoch)

In [13]:
set_seed(RANDOM_SEED)

folder_name = train(model, train_loader, valid_loader, criterion, optimizer, epoch, VAL_EVERY, folder_name)

Start training..
result model will be saved in /opt/ml/input/code/trained_model/[DeepLabV3Plus_efficientnet-b4]_[size:(512, 512)]_[loss:[(BCEWithLogitsLoss(), 0.5), (SmoothL1Loss(), 0.5)]]_[LR:0.0001]_[seed:21]_[epoch:50]


parameter/lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
sum/dice_coef,▁▁▁▂▅▅▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇████████████████
sum/train_loss,█▆▅▅▄▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
sum/valid loss,█▆▆▅▄▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
parameter/lr,0.0001
sum/dice_coef,0.93766
sum/train_loss,0.01324
sum/valid loss,0.01492


2023-06-09 21:57:20 | Epoch [1/50], Step [0/80]
2023-06-09 21:58:38 | Epoch [1/50], Step [25/80], Loss: 3.0925
2023-06-09 21:59:56 | Epoch [1/50], Step [50/80], Loss: 2.4369
2023-06-09 22:01:13 | Epoch [1/50], Step [75/80], Loss: 1.7549
Start validation # 1



finger-1    : 0.0000
finger-2    : 0.0000
finger-3    : 0.0000
finger-4    : 0.0000
finger-5    : 0.0000
finger-6    : 0.0000
finger-7    : 0.0006
finger-8    : 0.0000
finger-9    : 0.0000
finger-10   : 0.0000
finger-11   : 0.0022
finger-12   : 0.0000
finger-13   : 0.0000
finger-14   : 0.0000
finger-15   : 0.0001
finger-16   : 0.0000
finger-17   : 0.0000
finger-18   : 0.0000
finger-19   : 0.0000
Trapezium   : 0.0000
Trapezoid   : 0.0000
Capitate    : 0.0000
Hamate      : 0.0000
Scaphoid    : 0.0000
Lunate      : 0.0000
Triquetrum  : 0.0012
Pisiform    : 0.0000
Radius      : 0.0207
Ulna        : 0.0096
Best performance at epoch: 1, 0.0000 -> 0.0012
Save model in /opt/ml/input/code/trained_model/[DeepLabV3Plus_efficientnet-b4]_[size:(512, 512)]_[loss:[(BCEWithLogitsLoss(), 0.5), (SmoothL1Loss(), 0.5)]]_[LR:0.0001]_[seed:21]_[epoch:50]
2023-06-09 22:02:34 | Epoch [2/50], Step [0/80]
2023-06-09 22:03:52 | Epoch [2/50], Step [25/80], Loss: 1.4739
2023-06-09 22:05:09 | Epoch [2/50], Step [5


finger-1    : 0.0000
finger-2    : 0.0000
finger-3    : 0.0010
finger-4    : 0.0000
finger-5    : 0.0000
finger-6    : 0.0000
finger-7    : 0.0000
finger-8    : 0.0000
finger-9    : 0.0000
finger-10   : 0.0000
finger-11   : 0.0023
finger-12   : 0.0000
finger-13   : 0.0000
finger-14   : 0.0000
finger-15   : 0.0000
finger-16   : 0.0000
finger-17   : 0.0000
finger-18   : 0.0005
finger-19   : 0.0001
Trapezium   : 0.0000
Trapezoid   : 0.0000
Capitate    : 0.0000
Hamate      : 0.0000
Scaphoid    : 0.0000
Lunate      : 0.0000
Triquetrum  : 0.0000
Pisiform    : 0.0000
Radius      : 0.0017
Ulna        : 0.0038
2023-06-09 22:07:57 | Epoch [3/50], Step [0/80]
2023-06-09 22:09:14 | Epoch [3/50], Step [25/80], Loss: 1.1325
2023-06-09 22:10:30 | Epoch [3/50], Step [50/80], Loss: 1.0457
2023-06-09 22:11:47 | Epoch [3/50], Step [75/80], Loss: 0.9839
Start validation # 3


...


finger-1    : 0.0502
finger-2    : 0.7638
finger-3    : 0.7420
finger-4    : 0.1246
finger-5    : 0.7270
finger-6    : 0.7190
finger-7    : 0.7772
finger-8    : 0.2830
finger-9    : 0.7403
finger-10   : 0.7733
finger-11   : 0.7846
finger-12   : 0.0764
finger-13   : 0.6569
finger-14   : 0.7331
finger-15   : 0.7597
finger-16   : 0.0080
finger-17   : 0.0000
finger-18   : 0.7601
finger-19   : 0.7359
Trapezium   : 0.6654
Trapezoid   : 0.7018
Capitate    : 0.7510
Hamate      : 0.7276
Scaphoid    : 0.6892
Lunate      : 0.7166
Triquetrum  : 0.7297
Pisiform    : 0.6998
Radius      : 0.8631
Ulna        : 0.7982
Best performance at epoch: 46, 0.5960 -> 0.6054
Save model in /opt/ml/input/code/trained_model/[DeepLabV3Plus_efficientnet-b4]_[size:(512, 512)]_[loss:[(BCEWithLogitsLoss(), 0.5), (SmoothL1Loss(), 0.5)]]_[LR:0.0001]_[seed:21]_[epoch:50]
2023-06-10 01:57:54 | Epoch [47/50], Step [0/80]
2023-06-10 01:59:10 | Epoch [47/50], Step [25/80], Loss: 0.2976
2023-06-10 02:00:27 | Epoch [47/50], Ste


finger-1    : 0.1010
finger-2    : 0.7608
finger-3    : 0.7431
finger-4    : 0.1525
finger-5    : 0.7333
finger-6    : 0.7214
finger-7    : 0.7792
finger-8    : 0.3263
finger-9    : 0.7382
finger-10   : 0.7758
finger-11   : 0.7864
finger-12   : 0.1137
finger-13   : 0.7114
finger-14   : 0.7328
finger-15   : 0.7603
finger-16   : 0.0101
finger-17   : 0.0008
finger-18   : 0.7538
finger-19   : 0.7311
Trapezium   : 0.6629
Trapezoid   : 0.7029
Capitate    : 0.7517
Hamate      : 0.7262
Scaphoid    : 0.6898
Lunate      : 0.7287
Triquetrum  : 0.7222
Pisiform    : 0.7090
Radius      : 0.8641
Ulna        : 0.8033
Best performance at epoch: 47, 0.6054 -> 0.6135
Save model in /opt/ml/input/code/trained_model/[DeepLabV3Plus_efficientnet-b4]_[size:(512, 512)]_[loss:[(BCEWithLogitsLoss(), 0.5), (SmoothL1Loss(), 0.5)]]_[LR:0.0001]_[seed:21]_[epoch:50]
2023-06-10 02:03:02 | Epoch [48/50], Step [0/80]
2023-06-10 02:04:20 | Epoch [48/50], Step [25/80], Loss: 0.2975
2023-06-10 02:05:36 | Epoch [48/50], Ste


finger-1    : 0.1771
finger-2    : 0.7574
finger-3    : 0.7451
finger-4    : 0.1801
finger-5    : 0.7376
finger-6    : 0.7246
finger-7    : 0.7807
finger-8    : 0.3719
finger-9    : 0.7370
finger-10   : 0.7783
finger-11   : 0.7881
finger-12   : 0.1531
finger-13   : 0.7329
finger-14   : 0.7333
finger-15   : 0.7612
finger-16   : 0.0130
finger-17   : 0.0137
finger-18   : 0.7475
finger-19   : 0.7279
Trapezium   : 0.6613
Trapezoid   : 0.7016
Capitate    : 0.7526
Hamate      : 0.7252
Scaphoid    : 0.6899
Lunate      : 0.7366
Triquetrum  : 0.7122
Pisiform    : 0.7158
Radius      : 0.8653
Ulna        : 0.8081
Best performance at epoch: 48, 0.6135 -> 0.6217
Save model in /opt/ml/input/code/trained_model/[DeepLabV3Plus_efficientnet-b4]_[size:(512, 512)]_[loss:[(BCEWithLogitsLoss(), 0.5), (SmoothL1Loss(), 0.5)]]_[LR:0.0001]_[seed:21]_[epoch:50]
2023-06-10 02:08:14 | Epoch [49/50], Step [0/80]
2023-06-10 02:09:31 | Epoch [49/50], Step [25/80], Loss: 0.2975
2023-06-10 02:10:47 | Epoch [49/50], Ste


finger-1    : 0.2660
finger-2    : 0.7537
finger-3    : 0.7466
finger-4    : 0.2096
finger-5    : 0.7390
finger-6    : 0.7273
finger-7    : 0.7827
finger-8    : 0.4197
finger-9    : 0.7367
finger-10   : 0.7792
finger-11   : 0.7907
finger-12   : 0.1926
finger-13   : 0.7430
finger-14   : 0.7351
finger-15   : 0.7622
finger-16   : 0.0164
finger-17   : 0.0523
finger-18   : 0.7420
finger-19   : 0.7264
Trapezium   : 0.6609
Trapezoid   : 0.7019
Capitate    : 0.7544
Hamate      : 0.7260
Scaphoid    : 0.6907
Lunate      : 0.7422
Triquetrum  : 0.7050
Pisiform    : 0.7194
Radius      : 0.8667
Ulna        : 0.8129
Best performance at epoch: 49, 0.6217 -> 0.6311
Save model in /opt/ml/input/code/trained_model/[DeepLabV3Plus_efficientnet-b4]_[size:(512, 512)]_[loss:[(BCEWithLogitsLoss(), 0.5), (SmoothL1Loss(), 0.5)]]_[LR:0.0001]_[seed:21]_[epoch:50]
2023-06-10 02:13:23 | Epoch [50/50], Step [0/80]
2023-06-10 02:14:41 | Epoch [50/50], Step [25/80], Loss: 0.2974
2023-06-10 02:15:58 | Epoch [50/50], Ste


finger-1    : 0.3621
finger-2    : 0.7484
finger-3    : 0.7476
finger-4    : 0.2397
finger-5    : 0.7388
finger-6    : 0.7292
finger-7    : 0.7845
finger-8    : 0.4690
finger-9    : 0.7356
finger-10   : 0.7794
finger-11   : 0.7929
finger-12   : 0.2329
finger-13   : 0.7497
finger-14   : 0.7357
finger-15   : 0.7612
finger-16   : 0.0202
finger-17   : 0.1251
finger-18   : 0.7356
finger-19   : 0.7236
Trapezium   : 0.6607
Trapezoid   : 0.7004
Capitate    : 0.7568
Hamate      : 0.7261
Scaphoid    : 0.6896
Lunate      : 0.7466
Triquetrum  : 0.6978
Pisiform    : 0.7217
Radius      : 0.8679
Ulna        : 0.8168
Best performance at epoch: 50, 0.6311 -> 0.6412
Save model in /opt/ml/input/code/trained_model/[DeepLabV3Plus_efficientnet-b4]_[size:(512, 512)]_[loss:[(BCEWithLogitsLoss(), 0.5), (SmoothL1Loss(), 0.5)]]_[LR:0.0001]_[seed:21]_[epoch:50]


In [14]:
#inference(folder_name, A.Resize(resize, resize))